In [1]:
from urllib.request import Request, urlopen, addinfourl
from bs4 import BeautifulSoup
import re
import csv
import time

# function to create and append data to csv file , which takes the list of rows to append to .csv file
def writeinfile(list):
    csvFile = open('meroFile.csv', 'a+')
    try:
        writer = csv.writer(csvFile)
        writer.writerow(('Title','Place','Land Size','PublishDate','Price')) #Headings for csv file
        for i in range(0, len(list), 6):
            writer.writerow((list[i], list[i+1], list[i+2], list[i+3], list[i+4]))
    finally:
        csvFile.close()

# function to get contents from website. Takes url of the page as argument       
def getContents(pageUrl):
    request = Request(f"https://hamrobazaar.com/search.php{pageUrl}")
    request.add_header("User-Agent", "*") # to prevent httpError, User-agent is to be defined.

    html = urlopen(request)
    bs = BeautifulSoup(html.read(), 'lxml')
    content_list = list()
    for table in bs.find_all('table'):
        if len(list(table.tr.children)) == 13:
            if table.center:
                contents = table.tr
                titles = contents.find_all('font',style='font-size:13px;')
                place_landsize = contents.find_all('font',color='#565d60')
                post_date = contents.find_all('td',width='85')
                prices = contents.find_all('td',width='100')
                
                # add title, place, landsize, postdate, prices to content_list first then append that list to csv file.
                for title in titles:
                    content_list.append(title.b.text)
                    # print(title.b.text)
                    
                for psize in place_landsize:
                    totaltext = psize.b.i.text
                    textlist = totaltext.split('|')
                    if len(textlist) == 0:
                        landSize = 0
                        place = " "
                    elif len(textlist) == 1:
                        place = textlist[0]
                        landSize = " "
                    else:
                        place = textlist[0]
                        landSize = textlist[-1]
                    content_list.append(place)
                    content_list.append(landSize)
                    # print(place,landSize, sep='\n')
                    
                for pdate in post_date:
                    content_list.append(pdate.text)
                    # print(pdate.text)
                    
                for price in prices:
                    content_list.append(price.b.text)
                    # print(price.b.text)
                
                # print("-"*100)
                content_list.append("\n")
    return content_list
      
                            
pages = list() # contains all the links to be traversed from hamrobazaar site.
# function which parses all the relevant links and appends to pages list.
def getLinks(pageUrl):
    global pages
    request = Request("https://hamrobazaar.com/search.php?do_search=Search&searchword=land")
    request.add_header("User-Agent", "*")

    html = urlopen(request)
    bs = BeautifulSoup(html.read(), 'lxml')
    
    for link in bs.find_all('a', href=re.compile('(offset=\d+)$')):
        if 'href' in link.attrs:
            if link['href'] not in pages:
                # We have encountered a new page
                newPage = link['href']
                print(newPage)
                pages.append(newPage)
                getLinks(newPage)
    
getLinks('')
pagesCopyOriginal = pages.copy()

# since page links follow pattern with each link differs by ".....offset={i}", where i is incremented by 20 every page.
# if first page has link "....offset=20, 2nd page has link "offset=40" and so on.
for i in range(20, int(pages[-1].split('=')[-1]) + 1, 20):
    link = f"?&do_search=1&searchword=land&order=popularad&offset={i}"
    if link not in pages:
        pages.append(link)

# main control flow
for page in pages:
    contentList = getContents(page) # retrieves all contents from a page in form of list
    writeinfile(contentList) # appends above contents present in list in csv file
    print("Delay"*20) # output to console to see if the programming is running
    time.sleep(25) # sleep for 25sec to prevent server overload

?&do_search=1&searchword=land&order=popularad&offset=20
?&do_search=1&searchword=land&order=popularad&offset=40
?&do_search=1&searchword=land&order=popularad&offset=60
?&do_search=1&searchword=land&order=popularad&offset=80
?&do_search=1&searchword=land&order=popularad&offset=100
?&do_search=1&searchword=land&order=popularad&offset=120
?&do_search=1&searchword=land&order=popularad&offset=140
?&do_search=1&searchword=land&order=popularad&offset=160
?&do_search=1&searchword=land&order=popularad&offset=180
?&do_search=1&searchword=land&order=popularad&offset=200
?&do_search=1&searchword=land&order=popularad&offset=1780
?&do_search=1&searchword=land&order=popularad&offset=3780
?&do_search=1&searchword=land&order=popularad&offset=5580
DelayDelayDelayDelayDelayDelayDelayDelayDelayDelayDelayDelayDelayDelayDelayDelayDelayDelayDelayDelay
DelayDelayDelayDelayDelayDelayDelayDelayDelayDelayDelayDelayDelayDelayDelayDelayDelayDelayDelayDelay


KeyboardInterrupt: 